In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import logging

# ===============================
# הגדרות
# ===============================
BASE_TAG_DIR = r"tag"
OUTPUT_EXCEL = os.path.join(BASE_TAG_DIR, "annotation_overlap.xlsx")

ANNOTATORS = {
    "Hila": {"suffix": "_hs"},
    "Lital": {"suffix": "_l"},
    "Mustafa": {"suffix": "_m"},
}

LAB_PAIRS = [
    ("lab1", "Hila", "Lital"),
    ("lab2", "Lital", "Mustafa"),
    ("lab3", "Hila", "Mustafa"),
]

IMAGE_EXTENSIONS = (".jpg", ".jpeg", ".png", ".tif", ".tiff")

logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")

# ===============================
# פונקציות עזר
# ===============================
def normalize_name(filename, suffix):
    name = os.path.splitext(filename)[0].lower()
    if name.endswith(suffix):
        name = name[: -len(suffix)]
    return name

def load_mask(path):
    mask = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    if mask is None:
        raise ValueError(f"Cannot read image: {path}")
    return (mask > 0).astype(np.uint8)

def compute_iou(mask1, mask2):
    intersection = np.logical_and(mask1, mask2).sum()
    union = np.logical_or(mask1, mask2).sum()
    if union == 0:
        return 100.0
    return (intersection / union) * 100

def get_masks_map(folder, suffix):
    masks = {}
    for f in os.listdir(folder):
        if f.lower().endswith(IMAGE_EXTENSIONS):
            key = normalize_name(f, suffix)
            masks[key] = os.path.join(folder, f)
    return masks

# ===============================
# לוגיקה ראשית
# ===============================
rows = []

for lab, ann1, ann2 in LAB_PAIRS:
    logging.info(f"Processing {lab}: {ann1} vs {ann2}")

    dir1 = os.path.join(BASE_TAG_DIR, ann1, lab)
    dir2 = os.path.join(BASE_TAG_DIR, ann2, lab)

    if not os.path.isdir(dir1) or not os.path.isdir(dir2):
        logging.warning(f"Skipping {lab} – missing folder")
        continue

    masks1 = get_masks_map(dir1, ANNOTATORS[ann1]["suffix"])
    masks2 = get_masks_map(dir2, ANNOTATORS[ann2]["suffix"])

    all_keys = sorted(set(masks1.keys()) | set(masks2.keys()))

    for key in all_keys:
        if key not in masks1 or key not in masks2:
            rows.append({
                "lab": lab,
                "image_name": key,
                "annotator_1": ann1,
                "annotator_2": ann2,
                "similarity_percent": "No match"
            })
            continue

        try:
            mask1 = load_mask(masks1[key])
            mask2 = load_mask(masks2[key])

            if mask1.shape != mask2.shape:
                logging.warning(f"Resolution mismatch: {lab}/{key}")
                continue

            iou = compute_iou(mask1, mask2)

            rows.append({
                "lab": lab,
                "image_name": key,
                "annotator_1": ann1,
                "annotator_2": ann2,
                "similarity_percent": round(iou, 2)
            })

        except Exception as e:
            logging.warning(f"Failed processing {lab}/{key}: {e}")

# ===============================
# שמירה לאקסל
# ===============================
df = pd.DataFrame(rows)
df.to_excel(OUTPUT_EXCEL, index=False)

logging.info(f"Excel saved to: {OUTPUT_EXCEL}")
